In [ ]:
'''
Beta distribution neural network with 4 hidden dense layers 
and 2 beta distribution parameter layers to predict output distribution.
Preprocessing code written by Prof. Horvat, network code written by Meal Swipes for Late Days.
Input is a (batch_size x 26) tensor representing the input wave spectrum, 
output is a (batch size x 49) tensor representing the output floe size distribution.
'''
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.io import loadmat,savemat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
import tensorflow_probability as tfp
import random
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import drive 

In [ ]:
# Define file locations

usable_class = 0

include_last = True;

drive.mount('/content/drive', force_remount=True)
# locstr = '/Users/chorvat/Dropbox (Brown)/Research Projects/Active/FSTD-Ongoing-Work/Neural-Net-Waves/'
locstr = '/content/drive/Shared drives/DeepWIFF/'

training_string = locstr + 'Data/training_data_errtol'

if not include_last:
    training_string = training_string + '_nolast'

training_string = training_string + '.mat'


Mounted at /content/drive


In [ ]:
'''
Loads training data and performs preprocessing to remove degenerate inputs.
There is no need to run this cell once the pickle files have been saved once.
'''

training_data = loadmat(training_string)
misc_data = loadmat(locstr + 'Data/NN_params.mat')

input_vec = training_data['input_vec'].T
output_vec = training_data['output_vec'].T

Freq = misc_data['f']
dFreq = Freq * (np.sqrt(1.1) - np.sqrt(1/1.1)); 
Redge = misc_data['redge'].T
Rcent = misc_data['rcent'].T
Lambda = misc_data['lambda']


num_frac = np.sum(output_vec,axis=1)
# classifier = load_model('Classifiers/KERAS-Classifier-25x25.h5')
# thresh = 0.11;
# y_pred_class = classifier.predict(input_vec)
# usable = y_pred_class[:,1] > thresh

usable = num_frac > 0
print('Out of ' + str(len(num_frac)) + ', ' + str(sum(usable)) + ' can be used')

ice_thick = input_vec[:,-1]
wave_spec = input_vec[:,:25]

# wave_spec[wave_spec < 10**-8] = 0
wave_energy = np.sum(wave_spec*np.expand_dims(dFreq,axis=1).T,axis=1)
peak_loc = np.argmax(wave_spec,axis=1)
peak_freq = Freq[peak_loc]

NN_input = input_vec[usable]
NN_output = output_vec[usable]



Out of 1830900, 404503 can be used


In [ ]:
'''
Loads training and testing data from pickle file.
'''
import pickle

try:

    X_train = pickle.load( open(locstr +  "Data/" + "X_train.p", "rb" ) )
    X_test = pickle.load( open( locstr + "Data/" + "X_test.p", "rb" ) )
    y_train = pickle.load( open(locstr + "Data/" + "y_train.p","rb"))
    y_test = pickle.load( open(locstr + "Data/" + "y_test.p","rb"))
    
except:
        
        print('exception in loading segmented data')
        X_train, X_test, y_train, y_test = train_test_split(NN_input,NN_output,test_size=0.3)
        
        pickle.dump(X_train, open( locstr + "Data/" + "X_train.p", "wb" ) )
        pickle.dump(X_test, open( locstr + "Data/" + "X_test.p", "wb" ) )
        pickle.dump(y_train, open( locstr + "Data/" + "y_train.p", "wb" ) )
        pickle.dump(y_test, open( locstr + "Data/" + "y_test.p", "wb" ) )

In [ ]:
tf.print("X_train mean: ", tf.math.reduce_mean(X_train))
tf.print("X_train stdev: ", tf.math.reduce_std(X_train))
tf.print("y_train mean: ", tf.math.reduce_mean(y_train))
tf.print("y_train stdev: ", tf.math.reduce_std(y_train))

X_train mean:  1.1930075366304653
X_train stdev:  4.3379647876911962
y_train mean:  0.020408163265306121
y_train stdev:  0.066589406428670186


In [ ]:
tf.keras.backend.set_floatx('float64')

class BetaModel(tf.keras.Model):
    '''
    This model was an attempt to introduce a nondeterministic element into the network output. Each of the output bins is computed as a sample from an independent beta distribution parameterized by the network layers
    '''
    
    def __init__(self, num_bins):
        super(BetaModel, self).__init__()
        self.num_bins = num_bins
        # Training hyperparameters
        self.batch_size = 256
        self.learning_rate = .001
        self.optimizer = tf.keras.optimizers.Adam(learning_rate = self.learning_rate)

        # Dense layer hyperparameters
        self.hidden_architecture = [100, 100, 100, 100]
        self.activation = tf.keras.layers.LeakyReLU(alpha=0.2)
        self.regularizer = {} #{'kernel_regularizer': keras.regularizers.l2(0.001), 'bias_regularizer': keras.regularizers.l2(0.001)}


        # Hidden layers
        self.hidden_layers = [keras.layers.Dense(size, activation = self.activation, **self.regularizer) for size in self.hidden_architecture]
        # Beta distribution parameters
        self.alpha = keras.layers.Dense(num_bins, activation= 'softplus', **self.regularizer)
        self.beta = keras.layers.Dense(num_bins, activation= 'softplus', **self.regularizer)
        # Softmax for final output
        self.softmax = keras.layers.Softmax()



    def call(self, input):
        # Compute all hidden layer outputs
        layer_out = input
        for layer in self.hidden_layers:
            layer_out = layer(layer_out)
        # Compute distribution parameters
        alpha = self.alpha(layer_out)
        beta = self.beta(layer_out)
        # Initialize the the beta distributions
        dists = [tfp.distributions.Beta(alpha[:, bin], beta[:, bin], allow_nan_stats=False) for bin in range(self.num_bins)]
        # Sample each distribution and transpose to producean output tensor of shape (batch_size, num_bins)
        sampled = tf.transpose(tf.convert_to_tensor([dist.sample() for dist in dists]))
        # Softmax the sampled vector to ensure that the output is a well defined distribution
        return self.softmax(sampled)

In [ ]:
#Creating the Beta Distribution model
BETA = BetaModel(49)

"""
Compiles all the data for training and testing the model. Creates a list of different loss functions to keep track of.
"""
losses = ["mse", "kld", "mae"]
BETA.compile(optimizer = BETA.optimizer, loss = losses[0], metrics = ["kullback_leibler_divergence","mean_squared_error", "mean_absolute_error"])

# Output name
outstr = locstr + 'Network Weights/BETA'

if not include_last:
    outstr = outstr + '-nolast'        

#Adds early stopping, if so desired
es = EarlyStopping(monitor='val_loss', verbose=1, patience=10);

# Reduce learning rate on plateau
lradj = keras.callbacks.ReduceLROnPlateau()

#Initializing necessary TensorBoard metrics for better metrics
tb_log = tf.keras.callbacks.TensorBoard(
    log_dir = locstr + "logs",
    histogram_freq = 5,
    write_graph = True,
    update_freq= 50,
    profile_batch=2,
    embeddings_freq=1)

"""
Trains and tests the Beta Distribution network. Validation_data = data to be tested. Automatically trains and tests for 100 epochs
"""
history = BETA.fit(X_train,y_train, validation_data = (X_test,y_test), verbose = 1, batch_size = BETA.batch_size, epochs=100, callbacks = [lradj])

#Saving the weights of the Beta model     
#BETA.save(outstr, save_format='tf')

Epoch 1/100
1107/1107 [==============================] - 42s 38ms/step - loss: 0.0037 - kullback_leibler_divergence: 1.6542 - mean_squared_error: 0.0037 - mean_absolute_error: 0.0315 - val_loss: 0.0036 - val_kullback_leibler_divergence: 1.6350 - val_mean_squared_error: 0.0036 - val_mean_absolute_error: 0.0314
Epoch 2/100
1107/1107 [==============================] - 39s 35ms/step - loss: 0.0037 - kullback_leibler_divergence: 1.6281 - mean_squared_error: 0.0037 - mean_absolute_error: 0.0314 - val_loss: 0.0036 - val_kullback_leibler_divergence: 1.6112 - val_mean_squared_error: 0.0036 - val_mean_absolute_error: 0.0312
Epoch 3/100
1107/1107 [==============================] - 39s 35ms/step - loss: 0.0036 - kullback_leibler_divergence: 1.6138 - mean_squared_error: 0.0036 - mean_absolute_error: 0.0313 - val_loss: 0.0036 - val_kullback_leibler_divergence: 1.6091 - val_mean_squared_error: 0.0036 - val_mean_absolute_error: 0.0312
Epoch 4/100
1107/1107 [==============================] - 39s 35ms/s

In [ ]:
print(history.history)
pickle.dump(history.history, open( locstr + "Data/" + "BETA-100x100x100x100x10-MSE-FAKE.p", "wb" ))
plt.plot(log(history.history['loss']))
plt.plot(log(history.history['val_loss']))
np.min(history.history['val_loss'])

In [ ]:
#For assessing network performance, load a saved model.
model_name = 'Full-100x100x100x100x100-mse-FINAL.h5'
BETA = load_model(locstr + "Network Weights/Horvat Checkpoints/" + model_name)

In [ ]:
eval = BETA.evaluate(X_test, return_dict = True)
for name in BETA.metrics_names:
    print("%s: %f" % (name, eval[name]))

In [ ]:
# q = random.randint(0,y_test.shape[0])
# plt.semilogx(Rcent,y_test[q,:])

# test_out = BETA.predict(np.expand_dims(X_test[q,:],axis=1).T).T

# plt.semilogx(Rcent,test_out)

# print('MAE value here is ' + str(sum(abs(np.subtract(np.expand_dims(y_test[q,:],axis=1),test_out)))))

test_out = BETA.predict(X_test)

In [ ]:
errmat = np.sum(np.abs((test_out - y_test)),axis=1)
errmat2 = np.mean((test_out - y_test)**2,axis=1)
testE = np.sum(X_test[:,:25]*dFreq.T,axis=1)
testH = X_test[:,-1]

print(errmat.shape)
print(testE.shape)
print(testH.shape)

print('SAE is: ' + str(np.nanmean(errmat)))
print(">10% SAE is: " + str(np.sum(errmat > 0.1)/len(X_test)))
print('MSE is: ' + str(np.nanmean(errmat2)))


In [ ]:
feed_forward_network = load_model(locstr + "Network Weights/" + "Full-100x100x100x100x100-mse-FINAL_2.h5")